In [ ]:
import cv2
import numpy as np
import torch
from models import SINet
import torchvision.transforms as T


In [ ]:
torch.cuda.init()

model = SINet()

state_dict = torch.load('checkpoints/best_model.pt')
model.load_state_dict(state_dict)


model = model.cuda()



In [ ]:
cap = cv2.VideoCapture('movie.mov')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

res=(int(width), int(height))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter('./output.mp4', fourcc, 20.0, res)


model.eval()

data_info = torch.load('datasets/data_info.pt')


ret, frame = cap.read()
while(ret):
    with torch.no_grad():

        transform = T.Compose([T.ToTensor(),
                               T.Normalize(data_info['mean'].reshape((3, 1, 1)) / 255.,
                                           data_info['std'].reshape((3, 1, 1)) / 255.)])

        img = transform(cv2.resize(frame, (224, 224))[..., ::-1].copy())

        img = img.cuda()

        m = model(img.unsqueeze(0))

        mask = m.squeeze().detach().cpu().numpy()>=0.5

        mask = cv2.resize(mask.astype(np.uint8), tuple(frame.shape[:2][::-1]))

        frame[mask==0, :]=np.array((128,0,255)).reshape((1,1,3))

        writer.write(frame)
    
    ret, frame = cap.read()
    
        
writer.release()
cap.release()
    